In [ ]:
import pystan 
import numpy as np
from sklearn.preprocessing import PolynomialFeatures 
import matplotlib.pyplot as plt
from scipy.stats import norm
import arviz # For visualization

In [ ]:
# Data-generating function
def f(x):
    return 10*x + 15
    
# Returns a noisy sample from f
def noisyF(x, sigma=0.1):
    return np.random.normal(f(x), sigma)
    
# Samples n_samples independent noisy samples from f where x is selected uniformly at random from interval
def sampleF(n_samples, interval=[0, 6], sigma=0.1):
    x = np.random.uniform(0, 6, n_samples)
    y = noisyF(x, sigma)
    return x, y

# Polynomial basis functions
def phi(x, degree=1):
    poly = PolynomialFeatures(degree=degree)
    return poly.fit_transform(x.reshape(-1,1))

In [ ]:
# Generate data
nr_samples = 50
noise_sigma = 5

np.random.seed(1)

(x_train_orig, y_train) = sampleF(nr_samples, sigma=noise_sigma)
(x_test_orig, y_test) = sampleF(nr_samples, sigma=noise_sigma)

poly = PolynomialFeatures(1)
x_train = np.matrix(poly.fit_transform(x_train_orig.reshape(-1, 1)))
x_test = np.matrix(poly.fit_transform(x_test_orig.reshape(-1, 1)))
y_train = np.matrix(y_train).T
y_test = np.matrix(y_test).T

plt.scatter(x_train_orig, y_train.tolist())
plt.show()

## Bayesian Linear regression

Assume that we have observed $n$ pairs $(x_i, y_i)$ where $x_i$ are the feature values and $y_i$ is the label. Let $w\in \mathbb{R}^d$ be our regression weights. The likelihood is 
$$P(y | x, w, \beta) = \prod_{i=1}^{n} N(y_i | w^Tx_i, \beta^{-1}).$$
The parameter $\beta$ has a Gamma prior
$$P(\beta) = Gamma(\beta | a_0, b_0)$$
where $a_0$ and $b_0$ are user-defined hyperparameters. The regression weights have a Gaussian prior
$$P(w) = N(w | \mathbf{0}, \alpha^{-1}\mathbf{I})$$
where $\alpha$ (prior precision) is a user-defined hyperparameter. $$$$

In [ ]:
# Specify a standard linear regression model using Stan
lr_std_code = """
data {
    int<lower=0> N;      // Number of observations
    int<lower=0> D;      // Number of dimensions
    vector[N] Y;         // Labels
    matrix[N, D] X;      // Data
    real<lower=0> a0;     // A hyperparameter for noise precision
    real<lower=0> b0;     // A hyperparameter for noise precision
    real<lower=0> alpha; // Prior precision
}
parameters {
    real<lower=0> beta;  // Noise precision
    vector[D] W;         // Weights
}
model {
    for (n in 1:N) {
        Y[n] ~ normal(X[n]*W, sqrt(1/beta)); // Likelihood
    }
    // Priors
    beta ~ gamma(a0, b0);
    W ~ normal(0, sqrt(1/alpha));
}
""" 

# Compile the model
lr_std = pystan.StanModel(model_code = lr_std_code)

In [ ]:
degree = 1

phi_train = phi(x_train_orig, degree=degree)

n = phi_train.shape[0]
d = phi_train.shape[1]

# Hyperparameters
alpha= 0.1
a0 = 0.01
b0 = 0.01

# Input data for sampler
# A dictionary: the keys and the types of items are described by the "data" block of your model
data_std = {'N': n,
           'D': d,
           'Y': np.array(y_train).reshape(n,),
           'X': phi_train,
           'a0': a0,
           'b0': b0,
           'alpha': alpha}

# Parameters that control the HMC sampler
params = {
    'max_treedepth': 10
}



In [ ]:
%%time
fit_std = lr_std.sampling(data=data_std, iter=2000, chains=4, control=params) # Sample from the posterior

In [ ]:
# Print a summary of inference results
# Includes the mean, standard error, standard deviation, and various percentiles. 
# n_eff can be used to assess mixing, Rhat for assessing convergence
# lp__ is the (unnormalized) log-posterior
print(fit_std)

In [ ]:
# Plot posterior marginals (left) and traceplots (right) 
arviz.plot_trace(fit_std)
plt.show()

In [ ]:
# Extract samples
fit_std.extract()

In [ ]:
def predict(x, w, beta, n_predictions=1):
    n_samples = w_samples.shape[0]
    preds = []
    for i in range(n_predictions):
        ind = np.random.randint(0, n_samples)
        m = np.dot(w[ind, :].reshape(-1, 1).T, x.T)
        sample = norm.rvs(loc=m, scale=np.sqrt(1/beta[ind]))
        preds.append(sample)
    return np.array(preds)

def credibleIntervalSamples(x, w_samples, beta_samples, ci, n_predictions=1000):
    preds =  predict(x, w_samples, beta_samples, n_predictions)

    preds_sorted = np.sort(preds)

    lb_ind = int(np.ceil((1 - ci)/2*n_predictions))
    lb = preds_sorted[lb_ind]

    ub_ind = int(np.floor((ci + (1 - ci)/2)*n_predictions))
    ub = preds_sorted[ub_ind]
    
    m = np.mean(preds, axis=0)
    
    return m, lb, ub

def plotCredibleIntervals(x_grid, w_samples, beta_samples, ci=0.95, n_predictions=1000):
    ms = []
    lbs = []
    ubs = []
    for xx in x_grid:
        m, lb, ub = credibleIntervalSamples(phi(np.array([xx]), degree=degree), w_samples, beta_samples, ci, n_predictions=n_predictions)
        ms.append(m)
        lbs.append(lb)
        ubs.append(ub)

    plt.scatter(x_train_orig, y_train.tolist())
    plt.plot(x_grid, ms)
    plt.plot(x_grid, lbs, c='r')
    plt.plot(x_grid, ubs, c='r')
    plt.show()

In [ ]:
# Plot credible intervals 
ci = 0.95
n_predictions = 1500

w_samples = fit_std.extract()['W']
beta_samples = fit_std.extract()['beta']

x_grid = np.linspace(-3, 7, 51)

plotCredibleIntervals(x_grid, w_samples, beta_samples, ci=ci, n_predictions=n_predictions)

## ARD regression

Assume that we have observed $n$ pairs $(x_i, y_i)$ where $x_i$ are the feature values and $y_i$ is the label. Let $w\in \mathbb{R}^d$ be our regression weights. The likelihood is 
$$P(y | x, w, \beta) = \prod_{i=1}^{n} N(y_i | w^Tx_i, \beta^{-1}).$$

For the noise precision $\beta$, we use a Gamma prior
$$ P(\beta) = Gamma(\beta | a_0, b_0) $$
where $a_0$ and $b_0$ are user-defined hyperparameters. 

The prior for the weights $w$ is a Gaussian distribution 
$$ P(w | \alpha_1, \ldots \alpha_d) = \prod_{j=1}^{d} N(w_j | 0, \alpha_j^{-1}). $$
Notice that the difference compared to the standard case is that instead of having one precision parameter $\alpha$ that is same for all dimensions, we have a separate prior precision $\alpha_j$ for each dimension. Furthermore, $\alpha_j$ is not assumed to be known but we place a prior on it. The prior for $\alpha_j$ is a Gamma distribution
$$ P(\alpha_j) =  Gamma(\alpha_j | c_0, d_0)\quad \forall j=1,\ldots, d$$
where $c_0$ and $d_0$ are user-defined hyperparameters.  

In [ ]:
lr_ard_code = """
data {
    int<lower=0> N;   // Number of observations
    int<lower=0> D;   // Number of dimensions
    matrix[N,D] X;    // Data
    vector[N] Y;      // Labels
    real<lower=0> a0; // A hyperparameter for noise precision
    real<lower=0> b0; // A hyperparameter for noise precision
    real<lower=0> c0; // A hyperparameter for prior precision (ARD)
    real<lower=0> d0; // A hyperparameter for prior precision (ARD)
    }    
parameters {
    vector[D] W;
    vector<lower=0>[D] alpha; // Prior precisions
    real<lower=0> beta; // Noise precision
    }
transformed parameters {
    vector<lower=0>[D] t_alpha; // Prior variances
    real<lower=0> t_beta; // Noise variance
    for (d in 1:D) {
        t_alpha[d] = 1/sqrt(alpha[d]);
        }
    t_beta =  1/sqrt(beta);
    }
model {
    beta ~ gamma(a0, b0);
    alpha ~ gamma(c0, d0);
    W ~ normal(0,  t_alpha);
    Y ~ normal(X*W, t_beta);
    }"""

In [ ]:
lr_ard = pystan.StanModel(model_code = lr_ard_code)

In [ ]:
degree = 1

phi_train = phi(x_train_orig, degree=degree)

n = phi_train.shape[0]
d = phi_train.shape[1]

# Hyperparameters
a0 = 0.01
b0 = 0.01
c0 = 0.01
d0 = 0.01

# Input data for sampler
# A dictionary: the keys and the types of items are described by the "data" block of your model
data_ard = {'N': n,
           'D': d,
           'Y': np.array(y_train).reshape(n,),
           'X': phi_train,
           'a0': a0,
           'b0': b0,
           'c0': c0,
           'd0': d0}

# Parameters that control the HMC sampler
params = {
    'max_treedepth': 10
}

In [ ]:
%%time
fit_ard = lr_ard.sampling(data=data_ard, iter=2000, chains=4, control=params) # Sample from the posterior

In [ ]:
print(fit_ard)

In [ ]:
# Plot posterior marginals (left) and traceplots (right) 
arviz.plot_trace(fit_ard)
plt.show()

In [ ]:
# Plot credible intervals 
ci = 0.95
n_predictions = 1000

w_samples = fit_ard.extract()['W']
beta_samples = fit_ard.extract()['beta']

x_grid = np.linspace(-3, 7, 51)

plotCredibleIntervals(x_grid, w_samples, beta_samples, ci=ci, n_predictions=n_predictions)